### Minimum Number of Flips to Convert Binary Matrix to Zero Matrix
Given a m x n binary matrix mat. In one step, you can choose one cell and flip it and all the four neighbours of it if they exist (Flip is changing 1 to 0 and 0 to 1). A pair of cells are called neighboors if they share one edge.

Return the minimum number of steps required to convert mat to a zero matrix or -1 if you cannot.

Binary matrix is a matrix with all cells equal to 0 or 1 only.

Zero matrix is a matrix with all cells equal to 0.

In [1]:
class Solution:
    def minFlips(self, mat) -> int:
        hashcode = 0; row = len(mat); col = len(mat[0])
        for i in range(row):
            for j in range(col):
                if mat[i][j] == 1:
                    k = i*col + j
                    hashcode |= 1<<k
                    
        if hashcode == 0:
            return 0
        
        queue = [hashcode]; seen = {hashcode}
        level = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                hashcode = queue.pop(0)
                if hashcode == 0:
                    return level
                for i in range(row):
                    for j in range(col):
                        new_hashcode = self.switch(i, j, row, col, hashcode)
                        if new_hashcode not in seen:
                            seen.add(new_hashcode)
                            queue.append(new_hashcode)
            level += 1
        return -1
        
    def switch(self, i, j, r, c, hashcode):
        for x, y in [(i,j), (i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            k = x*c + y
            if 0<=x<r and 0<=y<c:
                hashcode ^= 1<<k
        
        return hashcode

grid = [[1,1,1],[1,0,1],[0,0,0]]
Solution().minFlips(grid)

6

###  Shortest Path Visiting All Nodes
An undirected, connected graph of N nodes (labeled 0, 1, 2, ..., N-1) is given as graph.

graph.length = N, and j != i is in the list graph[i] exactly once, if and only if nodes i and j are connected.

Return the length of the shortest path that visits every node. You may start and stop at any node, you may revisit nodes multiple times, and you may reuse edges.

In [2]:
def shortestPathLength(graph) -> int:
    queue = []; seen = set()
    num_nodes = len(graph)
    for i in range(num_nodes):
        queue.append((i, 1<<i))
        seen.add((i, 1<<i))

    level = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            node, hashcode = queue.pop(0)
            if hashcode == 2**num_nodes-1:
                return level
            for i in graph[node]:
                if (i, hashcode | 1<<i) not in seen:
                    seen.add((i, hashcode|1<<i))
                    queue.append((i, hashcode|1<<i))
        level += 1
    
graph = [[1],[0,2,4],[1,3,4],[2],[1,2]]
shortestPathLength(graph)

4

### Bus Routes
We have a list of bus routes. Each routes[i] is a bus route that the i-th bus repeats forever. For example if routes[0] = [1, 5, 7], this means that the first bus (0-th indexed) travels in the sequence 1->5->7->1->5->7->1->... forever.

We start at bus stop S (initially not on a bus), and we want to go to bus stop T. Travelling by buses only, what is the least number of buses we must take to reach our destination? Return -1 if it is not possible.

In [10]:
from collections import defaultdict
class Solution:
    def numBusesToDestination(self, routes, S: int, T: int) -> int:
        if S == T: return 0
        hashmap = defaultdict(set)
        queue = []; seen = set()
        for route_index, route in enumerate(routes):
            for stop in route:
                hashmap[stop].add(route_index)
        
        for route_index in hashmap[S]:
            queue.append(route_index)
            seen.add(route_index)
        
        num_buses = 1
        while queue:
            size = len(queue)
            for _ in range(size):
                route_index = queue.pop(0)
                if route_index in hashmap[T]:
                    return num_buses
                for stop in routes[route_index]:
                    for next_index in hashmap[stop]:
                        if next_index not in seen:
                            seen.add(next_index)
                            queue.append(next_index)
            num_buses += 1
        return -1

routes = [[1, 2, 7], [3, 6, 7]]
S = 1
T = 6
Solution().numBusesToDestination(routes, S, T)

2

### Sliding Puzzle
On a 2x3 board, there are 5 tiles represented by the integers 1 through 5, and an empty square represented by 0.

A move consists of choosing 0 and a 4-directionally adjacent number and swapping it.

The state of the board is solved if and only if the board is [[1,2,3],[4,5,0]].

Given a puzzle board, return the least number of moves required so that the state of the board is solved. If it is impossible for the state of the board to be solved, return -1.

In [8]:
class Solution:
    def slidingPuzzle(self, board) -> int:
        target = [1,2,3,4,5,0]
        flat_board = []
        for i in range(len(board)):
            for j in range(len(board[0])):
                flat_board.append(board[i][j])
                if board[i][j] == 0:
                    zero_pos = (i,j)
        
        queue = [(flat_board, zero_pos)]
        seen = {tuple(flat_board)}
        level = 0
        
        while queue:
            size = len(queue)
            for _ in range(size):
                flat_board, zero_pos = queue.pop(0)
                if flat_board == target:
                    return level
                i, j = zero_pos
                for x,y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                    if 0<=x<len(board) and 0<=y<len(board[0]):
                        new_flat_board = self.swap(flat_board[:], i, j, x, y, board)
                        if tuple(new_flat_board) not in seen:
                            seen.add(tuple(new_flat_board))
                            queue.append((new_flat_board, (x,y)))
                
            level += 1
        return -1
    
    def swap(self, flat_board, i, j, x, y, board):
        p = i*len(board[0]) + j
        q = x*len(board[0]) + y
        flat_board[p], flat_board[q] = flat_board[q], flat_board[p]
        return flat_board

board = [[4,1,2],[5,0,3]]   
obj = Solution()
obj.slidingPuzzle(board)

5

### Find Eventual Safe States
In a directed graph, we start at some node and every turn, walk along a directed edge of the graph.  If we reach a node that is terminal (that is, it has no outgoing directed edges), we stop.

Now, say our starting node is eventually safe if and only if we must eventually walk to a terminal node.  More specifically, there exists a natural number K so that for any choice of where to walk, we must have stopped at a terminal node in less than K steps.

Which nodes are eventually safe?  Return them as an array in sorted order.

The directed graph has N nodes with labels 0, 1, ..., N-1, where N is the length of graph.  The graph is given in the following form: graph[i] is a list of labels j such that (i, j) is a directed edge of the graph.

In [4]:
class Solution:
    def eventualSafeNodes(self, graph):
        self.hashmap = {}
        ans = []
        for i in range(len(graph)):
            if i in self.hashmap and self.hashmap[i] == 'safe':
                ans.append(i)
            if i not in self.hashmap:
                if not self.is_unsafe(i, graph):
                    ans.append(i)
        return ans
    
    def is_unsafe(self, node, graph):
        self.hashmap[node] = 'unsafe'
        for neigh in graph[node]:
            if neigh in self.hashmap and self.hashmap[neigh] == 'unsafe':
                return True
            elif neigh not in self.hashmap and self.is_unsafe(neigh, graph):
                return True
        
        self.hashmap[node] = 'safe'
        return False

graph = [[1,2],[2,3],[5],[0],[5],[],[]]
obj = Solution()
obj.eventualSafeNodes(graph)

[2, 4, 5, 6]

### Bricks Falling When Hit
We have a grid of 1s and 0s; the 1s in a cell represent bricks.  A brick will not drop if and only if it is directly connected to the top of the grid, or at least one of its (4-way) adjacent bricks will not drop.

We will do some erasures sequentially. Each time we want to do the erasure at the location (i, j), the brick (if it exists) on that location will disappear, and then some other bricks may drop because of that erasure.

Return an array representing the number of bricks that will drop after each erasure in sequence.

In [2]:
class Solution:
    def hitBricks(self, grid, hits):
        for i, j in hits:
            grid[i][j] = 0 if grid[i][j] == 1 else -1
        
        for i in range(len(grid[0])):
            if grid[0][i] == 1:
                self.dfs(0, i, grid)
        
        ans = [0]*len(hits)
        
        for i in range(len(hits)-1, -1, -1):
            x, y = hits[i]
            grid[x][y] += 1
            if grid[x][y] == 1 and self.is_connected(x, y, grid):
                size = self.dfs(x, y, grid) - 1
                ans[i] = size
                
        return ans
    
    def is_connected(self, i, j, grid):
        if i == 0:
            return True
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 2:
                return True
        return False
    
    def dfs(self, i, j, grid):
        grid[i][j] = 2
        ans = 1
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 1:
                ans += self.dfs(x, y, grid)
        return ans
                
obj = Solution()
grid = [[1,0,0,0],[1,1,0,0]]
hits = [[1,1],[1,0]]
obj.hitBricks(grid, hits)

[0, 0]

### Race Car
Your car starts at position 0 and speed +1 on an infinite number line.  (Your car can go into negative positions.)

Your car drives automatically according to a sequence of instructions A (accelerate) and R (reverse).

When you get an instruction "A", your car does the following: position += speed, speed *= 2.

When you get an instruction "R", your car does the following: if your speed is positive then speed = -1 , otherwise speed = 1.  (Your position stays the same.)

For example, after commands "AAR", your car goes to positions 0->1->3->3, and your speed goes to 1->2->4->-1.

Now for some target position, say the length of the shortest sequence of instructions to get there.

In [13]:
import collections
def racecar(target: int) -> int:
    queue = collections.deque(); seen = {(0, 1)}
    queue.append((0,1))
    steps = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            pos, speed = queue.popleft()
            if pos == target:
                return steps

            # A
            new_pos = pos + speed
            new_speed = speed*2
            if (new_pos, new_speed) not in seen:
                seen.add((new_pos, new_speed))
                queue.append((new_pos, new_speed))

            # R
            new_speed = -1 if speed > 0 else 1
            if (pos, new_speed) not in seen:
                seen.add((pos, new_speed))
                queue.append((pos, new_speed))
        steps += 1

racecar(6)

5

### Most Stones Removed with Same Row or Column
On a 2D plane, we place stones at some integer coordinate points.  Each coordinate point may have at most one stone.

Now, a move consists of removing a stone that shares a column or row with another stone on the grid.

What is the largest possible number of moves we can make?

In [7]:
from collections import defaultdict
class Solution:
    def removeStones(self, stones) -> int:
        self.dic_row = defaultdict(list)
        self.dic_col = defaultdict(list)
        for x, y in stones:
            self.dic_row[x].append((x,y))
            self.dic_col[y].append((x,y))
        
        seen = set(); ans = 0
        for x, y in stones:
            if (x,y) not in seen:
                size = self.get_size(x, y, seen)
                ans += size-1
        return ans
    
    def get_size(self, i, j, seen):
        seen.add((i,j))
        size = 1
        
        for x, y in self.dic_row[i]:
            if (x,y) not in seen:
                size += self.get_size(x, y, seen)
        
        for x, y in self.dic_col[j]:
            if (x,y) not in seen:
                size += self.get_size(x, y, seen)
        
        return size

stones = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]]
Solution().removeStones(stones)

5

### Alphabet Board Path
On an alphabet board, we start at position (0, 0), corresponding to character board[0][0].

Here, board = ["abcde", "fghij", "klmno", "pqrst", "uvwxy", "z"], as shown in the diagram below.
We may make the following moves:

* 'U' moves our position up one row, if the position exists on the board;
* 'D' moves our position down one row, if the position exists on the board;
* 'L' moves our position left one column, if the position exists on the board;
* 'R' moves our position right one column, if the position exists on the board;
* '!' adds the character board[r][c] at our current position (r, c) to the answer.
* (Here, the only positions that exist on the board are positions with letters on them.)

Return a sequence of moves that makes our answer equal to target in the minimum number of moves.  You may return any path that does so.

 

In [15]:
def alphabetBoardPath(target: str) -> str:
    board = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'z']
    inverted_index = {}
    for i in range(len(board)):
        for j in range(len(board[i])):
            inverted_index[board[i][j]] = (i,j)

    queue = [(0, 0, 0, '')]

    while queue:
        i, j, idx, moves = queue.pop(0)
        if idx == len(target):
            return moves

        if target[idx] == 'z' and j != 0:
            moves += 'L'*(j-0)
            j = 0
        if target[idx] != 'z' and i == 5:
            moves += 'U'
            i = 4

        x, y = inverted_index[target[idx]]
        if x>i:
            moves += 'D'*(x-i)
        elif x<i:
            moves += 'U'*(i-x)

        if y>j:
            moves += 'R'*(y-j)
        elif y<j:
            moves += 'L'*(j-y)

        moves += '!'
        queue.append((x, y, idx+1, moves))

alphabetBoardPath('leet')

'DDR!UURRR!!DDD!'

### Delete Tree Nodes 
A tree rooted at node 0 is given as follows:

* The number of nodes is nodes;
* The value of the i-th node is value[i];
* The parent of the i-th node is parent[i].

Remove every subtree whose sum of values of nodes is zero.

After doing so, return the number of nodes remaining in the tree.

In [3]:
import collections
class Solution:
    def deleteTreeNodes(self, nodes: int, parent, value) -> int:
        graph = collections.defaultdict(list)
        for i, num in enumerate(parent):
            if num == -1:
                root = i
                continue
            graph[num].append(i)
        
        return self.dfs(root, graph, value)[1]
    
    def dfs(self, node, graph, value):
        val = value[node]; size = 1
        for child in graph[node]:
            sub_val, sub_size = self.dfs(child, graph, value)
            val += sub_val
            size += sub_size
        if val == 0:
            return 0, 0
        return val, size
    
nodes = 7; parent = [-1,0,0,1,2,2,2]; value = [1,-2,4,0,-2,-1,-1]    
obj = Solution()
obj.deleteTreeNodes(nodes, parent, value)

2

### Maximum Candies You Can Get from Boxes
Given n boxes, each box is given in the format [status, candies, keys, containedBoxes] where:

* status[i]: an integer which is 1 if box[i] is open and 0 if box[i] is closed.
* candies[i]: an integer representing the number of candies in box[i].
* keys[i]: an array contains the indices of the boxes you can open with the key in box[i].
* containedBoxes[i]: an array contains the indices of the boxes found in box[i].

You will start with some boxes given in initialBoxes array. You can take all the candies in any open box and you can use the keys in it to open new boxes and you also can use the boxes you find in it.

Return the maximum number of candies you can get following the rules above.

In [1]:
def maxCandies(status, candies, keys, containedBoxes, initialBoxes) -> int:
    visited = set()
    has_key = set()
    boxes = set()

    for box in initialBoxes:
        boxes.add(box)

    queue = []

    for box in boxes.copy():
        if status[box] == 1:
            queue.append(box)
            visited.add(box)
            boxes.remove(box)
    ans = 0

    while queue:
        size = len(queue)
        for _ in range(size):
            box = queue.pop(0)
            ans += candies[box]

            for key in keys[box]:
                has_key.add(key)

            for new_box in containedBoxes[box]:
                boxes.add(new_box)

        for box in boxes.copy():
            if ((box in has_key) or (status[box]==1)) and box not in visited:
                queue.append(box)
                visited.add(box)
                boxes.remove(box)

    return ans

status = [1,0,1,0]
candies = [7,5,4,100]
keys = [[],[],[1],[]]
containedBoxes = [[1,2],[3],[],[]] 
initialBoxes = [0]

maxCandies(status, candies, keys, containedBoxes, initialBoxes)

16

### Shortest Path to Get All Keys
We are given a 2-dimensional grid. "." is an empty cell, "#" is a wall, "@" is the starting point, ("a", "b", ...) are keys, and ("A", "B", ...) are locks.

We start at the starting point, and one move consists of walking one space in one of the 4 cardinal directions.  We cannot walk outside the grid, or walk into a wall.  If we walk over a key, we pick it up.  We can't walk over a lock unless we have the corresponding key.

For some 1 <= K <= 6, there is exactly one lowercase and one uppercase letter of the first K letters of the English alphabet in the grid.  This means that there is exactly one key for each lock, and one lock for each key; and also that the letters used to represent the keys and locks were chosen in the same order as the English alphabet.

Return the lowest number of moves to acquire all keys.  If it's impossible, return -1.

In [14]:
def shortestPathAllKeys(grid) -> int:
    hashcode = 0; num_keys = 0
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j].islower():
                num_keys += 1
            if grid[i][j] == '@':
                start = i, j

    seen = {(start[0], start[1], hashcode)}
    queue = [(start[0], start[1], hashcode)]
    ans = 0

    while queue:
        size = len(queue)
        for _ in range(size):
            i, j, hashcode = queue.pop(0)

            if grid[i][j].isupper():
                index = ord(grid[i][j]) - ord('A')
                if hashcode >> index & 1 == 0:
                    continue

            elif grid[i][j].islower():
                index = ord(grid[i][j]) - ord('a')
                hashcode = hashcode | 1 << index
                if hashcode == 2**num_keys-1:
                    return ans

            for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] != '#' and (x,y,hashcode) not in seen:
                    seen.add((x,y,hashcode))
                    queue.append((x,y,hashcode))
        ans += 1
    return -1

grid = ["@.a.#","###.#","b.A.B"]
shortestPathAllKeys(grid)

8

### Making A Large Island
In a 2D grid of 0s and 1s, we change at most one 0 to a 1.

After, what is the size of the largest island? (An island is a 4-directionally connected group of 1s).

In [1]:
from collections import Counter
class Solution:
    def largestIsland(self, grid) -> int:
        self.id = {}; num = 0; component_size = {}
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1 and (i,j) not in self.id:
                    size = self.dfs(i,j,grid, num)
                    component_size[num] = size
                    num += 1
        
        if len(self.id) == 0:
            return 1
        
        ans = max(component_size.values())
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 0:
                    neigh = set()
                    for x,y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                        if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 1:
                            neigh.add(self.id[(x,y)])
                    size = 0
                    for id in neigh:
                        size += component_size[id]
                    ans = max(ans, size+1)
        return ans
                             
    def dfs(self, i, j, grid, num):
        self.id[(i,j)] = num
        size = 1
        for x,y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 1 and (x,y) not in self.id:
                size += self.dfs(x, y, grid, num)
        return size
        
grid = [[1, 0], [0, 1]]
Solution().largestIsland(grid)

3

### Lexicographically Smallest Equivalent String
Given strings A and B of the same length, we say A[i] and B[i] are equivalent characters. For example, if A = "abc" and B = "cde", then we have 'a' == 'c', 'b' == 'd', 'c' == 'e'.

Equivalent characters follow the usual rules of any equivalence relation:

* Reflexivity: 'a' == 'a'
* Symmetry: 'a' == 'b' implies 'b' == 'a'
* Transitivity: 'a' == 'b' and 'b' == 'c' implies 'a' == 'c'
For example, given the equivalency information from A and B above, S = "eed", "acd", and "aab" are equivalent strings, and "aab" is the lexicographically smallest equivalent string of S.

Return the lexicographically smallest equivalent string of S by using the equivalency information from A and B.

In [9]:
from collections import defaultdict
from heapq import heappush
class Solution:
    def smallestEquivalentString(self, A: str, B: str, S: str) -> str:
        graph = defaultdict(list)
        for ch1, ch2 in zip(A, B):
            graph[ch1].append(ch2)
            graph[ch2].append(ch1)
        
        self.seen = {} #ch: id
        components = {} #id : comp
        num = 0
        for ch in graph.copy():
            if ch not in self.seen:
                comp = []
                self.dfs(ch, graph, comp, num)
                components[num] = comp
                num += 1
        
        res = ''
        for ch in S:
            if ch in self.seen:
                id_num = self.seen[ch]
                new_ch = components[id_num][0]
            else:
                new_ch = ch
            res += new_ch
        
        return res
                
    
    def dfs(self, ch, graph, cc, num):
        self.seen[ch] = num
        heappush(cc, ch)
        for neigh in graph[ch]:
            if neigh not in self.seen:
                self.dfs(neigh, graph, cc, num)

A = "leetcode"; B = "programs"; S = "sourcecode"
Solution().smallestEquivalentString(A, B, S)

'aauaaaaada'

### Number of Closed Islands 
Given a 2D grid consists of 0s (land) and 1s (water).  An island is a maximal 4-directionally connected group of 0s and a closed island is an island totally (all left, top, right, bottom) surrounded by 1s.

Return the number of closed islands.

**Similar Questions**
1. Surrounded Regions
2. Number of Enclaves

In [11]:
class Solution:
    def closedIsland(self, grid) -> int:
        rows = len(grid); cols = len(grid[0])
        outer = set(); inner = set()
        count = 0
        
        for i in range(cols):
            if grid[0][i] == 0 and (0, i) not in outer:
                self.dfs(0, i, grid, outer)
            if grid[rows-1][i] == 0 and (rows-1, i) not in outer:
                self.dfs(rows-1, i, grid, outer)
        
        for i in range(rows):
            if grid[i][0] == 0 and (i,0) not in outer:
                self.dfs(i, 0, grid, outer)
            if grid[i][cols-1] == 0 and (i, cols-1) not in outer:
                self.dfs(i, cols-1, grid, outer)
        
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 0 and (i,j) not in outer and (i,j) not in inner:
                    count += 1
                    self.dfs(i, j, grid, inner)
        
        return count
    
    def dfs(self, i, j, board, seen):
        seen.add((i, j))
        for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
            if 0<=x<len(board) and 0<=y<len(board[0]) and board[x][y] == 0 and (x,y) not in seen:
                self.dfs(x, y, board, seen)
        
grid = [[1,1,1,1,1,1,1,0],[1,0,0,0,0,1,1,0],[1,0,1,0,1,1,1,0],[1,0,0,0,0,1,0,1],[1,1,1,1,1,1,1,0]]
Solution().closedIsland(grid)

2

### Employee Importance
You are given a data structure of employee information, which includes the employee's unique id, his importance value and his direct subordinates' id.

For example, employee 1 is the leader of employee 2, and employee 2 is the leader of employee 3. They have importance value 15, 10 and 5, respectively. Then employee 1 has a data structure like [1, 15, [2]], and employee 2 has [2, 10, [3]], and employee 3 has [3, 5, []]. Note that although employee 3 is also a subordinate of employee 1, the relationship is not direct.

Now given the employee information of a company, and an employee id, you need to return the total importance value of this employee and all his subordinates.

In [6]:
# Definition for Employee.
class Employee:
    def __init__(self, id: int, importance: int, subordinates):
        self.id = id
        self.importance = importance
        self.subordinates = subordinates

class Solution:
    def getImportance(self, employees, id: int) -> int:
        emap = {}
        for emp in employees:
            emap[emp.id] = emp
        
        return self.helper(id, emap)
    
    def helper(self, id, emap):
        emp = emap[id]
        importance = emp.importance
        for sub in emp.subordinates:
            importance += self.helper(sub, emap)
        return importance

### 01 Matrix
Given a matrix consists of 0 and 1, find the distance of the nearest 0 for each cell.

The distance between two adjacent cells is 1.

In [10]:
from collections import deque
def updateMatrix(matrix):
    queue = deque(); visited = set()
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] == 0:
                queue.append((i,j))
                visited.add((i,j))

    dist = 0
    while queue:
        size = len(queue)
        for _ in range(size):
            i,j = queue.popleft()
            matrix[i][j] = dist

            for x,y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                if 0<=x<len(matrix) and 0<=y<len(matrix[0]) and (x,y) not in visited:
                    queue.append((x,y))
                    visited.add((x,y))
        dist += 1
    return matrix

matrix = [[0,0,0],[0,1,0],[1,1,1]]
updateMatrix(matrix)

[[0, 0, 0], [0, 1, 0], [1, 2, 1]]

###  As Far From Land As Possible
Given an N x N grid containing only values 0 and 1, where 0 represents water and 1 represents land, find a water cell such that its distance to the nearest land cell is maximized and return the distance.

The distance used in this problem is the Manhattan distance: the distance between two cells (x0, y0) and (x1, y1) is |x0 - x1| + |y0 - y1|.

If no land or water exists in the grid, return -1.

In [13]:
from collections import deque
def maxDistance(grid) -> int:
    queue = deque(); visited= set()
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 1:
                queue.append((i,j))
                visited.add((i,j))

    level = 0; ans = -1
    while queue:
        size = len(queue)
        for _ in range(size):
            i, j = queue.popleft()
            if grid[i][j] == 0:
                ans = level
            for x, y in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                if 0<=x<len(grid) and 0<=y<len(grid[0]) and grid[x][y] == 0 and (x,y) not in visited:
                    queue.append((x, y))
                    visited.add((x, y))
        level += 1
    return ans

grid = [[1,0,0],[0,0,0],[0,0,0]]
maxDistance(grid)

4

### Course Schedule IV
There are a total of n courses you have to take, labeled from 0 to n-1.

Some courses may have direct prerequisites, for example, to take course 0 you have first to take course 1, which is expressed as a pair: [1,0]

Given the total number of courses n, a list of direct prerequisite pairs and a list of queries pairs.

You should answer for each queries[i] whether the course queries[i][0] is a prerequisite of the course queries[i][1] or not.

Return a list of boolean, the answers to the given queries.

Please note that if course a is a prerequisite of course b and course b is a prerequisite of course c, then, course a is a prerequisite of course c.

In [6]:
from collections import defaultdict
class Solution:
    def checkIfPrerequisite(self, n: int, prerequisites, queries):
        graph = defaultdict(list)
        for x, y in prerequisites:
            graph[x].append(y)
        
        self.memo = {}; res = []
        for x, y in queries:
            self.helper(x, graph)
            if y in self.memo[x]:
                res.append(True)
            else:
                res.append(False)
        
        return res
    
    def helper(self, x, graph):
        if x in self.memo:
            return self.memo[x]
        children = set()
        for y in graph[x]:
            children = children | self.helper(y, graph) | {y}
        
        self.memo[x] = children
        return children

n = 5; prerequisites = [[0,1],[1,2],[2,3],[3,4]]; queries = [[0,4],[4,0],[1,3],[3,0]]
obj = Solution()
obj.checkIfPrerequisite(n, prerequisites, queries)

[True, False, True, False]

### Satisfiability of Equality Equations
Given an array equations of strings that represent relationships between variables, each string equations[i] has length 4 and takes one of two different forms: "a==b" or "a!=b".  Here, a and b are lowercase letters (not necessarily different) that represent one-letter variable names.

Return true if and only if it is possible to assign integers to variable names so as to satisfy all the given equations.

In [11]:
class Solution:
    def equationsPossible(self, equations) -> bool:
        uf = UnionFind()
        for x, op1, op2, y in equations:
            if op1 == '=':
                uf.union(ord(x)-ord('a'), ord(y)-ord('a'))
        
        for x, op1, op2, y in equations:
            if op1 == '!' and uf.find(ord(x)-ord('a')) == uf.find(ord(y)-ord('a')):
                return False
        return True
        
class UnionFind:
    def __init__(self):
        self.parent = list(range(26))
    
    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        if rootx == rooty:
            return False
        self.parent[rootx] = rooty
        return True
    
    def find(self, x):
        while x != self.parent[x]:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x
        
equations = ["a==b","b!=c","c==a"]
obj = Solution()
obj.equationsPossible(equations)

False

### Kth Ancestor of a Tree Node

You are given a tree with n nodes numbered from 0 to n-1 in the form of a parent array where parent[i] is the parent of node i. The root of the tree is node 0.

Implement the function getKthAncestor(int node, int k) to return the k-th ancestor of the given node. If there is no such ancestor, return -1.

The k-th ancestor of a tree node is the k-th node in the path from that node to the root.

In [10]:
from collections import defaultdict
class TreeAncestor:

    def __init__(self, n: int, parent):
        graph = defaultdict(list)
        self.ancestors = {}
        for child, p in enumerate(parent):
            if p == -1:
                root = child
            else:
                graph[p].append(child)
        
        self.dfs(root, graph, [])
    
    def dfs(self, root, graph, currPath):
        self.ancestors[root] = currPath
        for child in graph[root]:
            self.dfs(child, graph, currPath+[root])
            
    def getKthAncestor(self, node: int, k: int) -> int:
        ancestor_list = self.ancestors[node]
        if k > len(ancestor_list):
            return -1
        return ancestor_list[len(ancestor_list)-k]
    
obj = TreeAncestor(7,[-1,0,0,1,1,2,2])
obj.getKthAncestor(6, 2)

0

### Circular Array Loop
You are given a circular array nums of positive and negative integers. If a number k at an index is positive, then move forward k steps. Conversely, if it's negative (-k), move backward k steps. Since the array is circular, you may assume that the last element's next element is the first element, and the first element's previous element is the last element.

Determine if there is a loop (or a cycle) in nums. A cycle must start and end at the same index and the cycle's length > 1. Furthermore, movements in a cycle must all follow a single direction. In other words, a cycle must not consist of both forward and backward movements.

In [14]:
class Solution:
    def circularArrayLoop(self, nums) -> bool:
        state = {}
        for i in range(len(nums)):
            if i not in state and self.helper(i, state, nums):
                return True
        return False
    
    def helper(self, i, state, nums):
        state[i] = 'visiting'
        if nums[i] > 0:
            next_i = (i+nums[i]) % len(nums)
        else:
            next_i = (i+nums[i]+len(nums)) % len(nums)
        if next_i == i or nums[i]*nums[next_i] < 0:
            state[i] = 'visited'
            return False
        if next_i not in state and self.helper(next_i, state, nums):
            return True
        if state[next_i] == 'visiting':
            return True
        state[i] = 'visited'
        return False
            
Solution().circularArrayLoop([2,-1,1,2,2])

True

### Minimum Cost to Make at Least One Valid Path in a Grid

Given a m x n grid. Each cell of the grid has a sign pointing to the next cell you should visit if you are currently in this cell. The sign of grid[i][j] can be:
* 1 which means go to the cell to the right. (i.e go from grid[i][j] to grid[i][j + 1])
* 2 which means go to the cell to the left. (i.e go from grid[i][j] to grid[i][j - 1])
* 3 which means go to the lower cell. (i.e go from grid[i][j] to grid[i + 1][j])
* 4 which means go to the upper cell. (i.e go from grid[i][j] to grid[i - 1][j])

Notice that there could be some invalid signs on the cells of the grid which points outside the grid.

You will initially start at the upper left cell (0,0). A valid path in the grid is a path which starts from the upper left cell (0,0) and ends at the bottom-right cell (m - 1, n - 1) following the signs on the grid. The valid path doesn't have to be the shortest.

You can modify the sign on a cell with cost = 1. You can modify the sign on a cell one time only.

Return the minimum cost to make the grid have at least one valid path.

In [1]:
from heapq import heappush, heappop
class Solution:
    def minCost(self, grid) -> int:
        move = {1:lambda x,y: (x,y+1), 2:lambda x,y: (x,y-1), 3:lambda x,y: (x+1,y), 4:lambda x,y: (x-1,y)}
        heap = [(0,0,0)]
        seen = set()
        
        while heap:
            cost, x, y = heappop(heap)
            if x == len(grid)-1 and y == len(grid[0])-1:
                return cost
            if (x,y) in seen:
                continue
            seen.add((x,y))
            
            for k in [1,2,3,4]:
                i, j = move[k](x,y)
                c = 1 if k != grid[x][y] else 0
                if 0<=i<len(grid) and 0<=j<len(grid[0]) and (i,j) not in seen:
                    heappush(heap, (cost+c, i, j))
        
grid = [[1,1,1,1],[2,2,2,2],[1,1,1,1],[2,2,2,2]]
Solution().minCost(grid)

3

### Cut Off Trees for Golf Event

You are asked to cut off trees in a forest for a golf event. The forest is represented as a non-negative 2D map, in this map:

* 0 represents the obstacle can't be reached.
* 1 represents the ground can be walked through. The place with number bigger than 1 represents a tree can be walked through, and this positive number represents the tree's height.

In one step you can walk in any of the four directions top, bottom, left and right also when standing in a point which is a tree you can decide whether or not to cut off the tree.

You are asked to cut off all the trees in this forest in the order of tree's height - always cut off the tree with lowest height first. And after cutting, the original place has the tree will become a grass (value 1).

You will start from the point (0, 0) and you should output the minimum steps you need to walk to cut off all the trees. If you can't cut off all the trees, output -1 in that situation.

You are guaranteed that no two trees have the same height and there is at least one tree needs to be cut off.

In [8]:
from heapq import heappush, heappop
from collections import deque
class Solution:
    def cutOffTree(self, forest) -> int:
        heap = []
        for i in range(len(forest)):
            for j in range(len(forest[0])):
                if forest[i][j] > 0:
                    heappush(heap, (forest[i][j], i, j))
        
        curr_x, curr_y = 0, 0
        dist = 0
        while heap:
            h, x, y = heappop(heap)
            d = self.bfs(forest, curr_x, curr_y, x, y)
            if d == -1:
                return -1
            dist += d
            curr_x, curr_y = x, y
        return dist
    
    def bfs(self, grid, curr_x, curr_y, x, y):
        queue = deque()
        queue.append((curr_x, curr_y))
        seen = {(curr_x, curr_y)}
        d = 0
        while queue:
            size = len(queue)
            for _ in range(size):
                i, j = queue.popleft()
                if i == x and j == y:
                    return d
                for m, n in [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]:
                    if 0<=m<len(grid) and 0<=n<len(grid[0]) and grid[m][n]>0 and (m,n) not in seen:
                        seen.add((m, n))
                        queue.append((m,n))
            d += 1
        return -1
            
            
forest = [
 [1,2,3],
 [0,0,4],
 [7,6,5]
]

Solution().cutOffTree(forest)

6